In [10]:
import os, sys, random, shutil
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np


In [11]:
annotations = sorted(glob('/home/karush/code/karushp/project-helmet-det/helmet_det/raw_data/VOC2028/Annotations/*.xml'))


In [12]:
len(annotations)

7581

In [13]:
df = []
cnt = 0
for file in annotations:
  prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
  filename = str(cnt) + '.jpg'
  row = []
  parsedXML = ET.parse(file)
  for node in parsedXML.getroot().iter('object'):
    blood_cells = node.find('name').text
    xmin = int(node.find('bndbox/xmin').text)
    xmax = int(node.find('bndbox/xmax').text)
    ymin = int(node.find('bndbox/ymin').text)
    ymax = int(node.find('bndbox/ymax').text)

    row = [prev_filename, filename, blood_cells, xmin, xmax, ymin, ymax]
    df.append(row)
  cnt += 1

data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])


In [14]:
data[['prev_filename','filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('/home/karush/code/karushp/project-helmet-det/helmet_det/data/output.csv', index=False)
data.head(10)

,prev_filename,filename,cell_type,xmin,xmax,ymin,ymax
0,000000.jpg,0.jpg,hat,60,910,66,1108
1,000002.jpg,1.jpg,hat,37,76,32,84
2,000002.jpg,1.jpg,hat,97,125,44,82
3,000002.jpg,1.jpg,hat,165,208,103,158
4,000002.jpg,1.jpg,hat,178,213,71,113
5,000002.jpg,1.jpg,hat,221,251,44,88
6,000002.jpg,1.jpg,hat,249,283,61,112
7,000002.jpg,1.jpg,hat,335,376,60,112
8,000002.jpg,1.jpg,hat,344,385,107,163
9,000002.jpg,1.jpg,hat,372,402,59,110


In [15]:
img_width = 640
img_height = 480

def width(df):
  return int(df.xmax - df.xmin)
def height(df):
  return int(df.ymax - df.ymin)
def x_center(df):
  return int(df.xmin + (df.width/2))
def y_center(df):
  return int(df.ymin + (df.height/2))
def w_norm(df):
  return df/img_width
def h_norm(df):
  return df/img_height

df = pd.read_csv('/home/karush/code/karushp/project-helmet-det/helmet_det/data/output.csv')

le = preprocessing.LabelEncoder()
le.fit(df['cell_type'])
print(le.classes_)
labels = le.transform(df['cell_type'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center'].apply(w_norm)
df['width_norm'] = df['width'].apply(w_norm)

df['y_center_norm'] = df['y_center'].apply(h_norm)
df['height_norm'] = df['height'].apply(h_norm)

df.head(30)

['dog' 'hat' 'person']


,prev_filename,filename,cell_type,xmin,xmax,ymin,ymax,labels,width,height,x_center,y_center,x_center_norm,width_norm,y_center_norm,height_norm
0,000000.jpg,0.jpg,hat,60,910,66,1108,1,850,1042,485,587,0.757812,1.328125,1.222917,2.170833
1,000002.jpg,1.jpg,hat,37,76,32,84,1,39,52,56,58,0.087500,0.060937,0.120833,0.108333
2,000002.jpg,1.jpg,hat,97,125,44,82,1,28,38,111,63,0.173437,0.043750,0.131250,0.079167
3,000002.jpg,1.jpg,hat,165,208,103,158,1,43,55,186,130,0.290625,0.067187,0.270833,0.114583
4,000002.jpg,1.jpg,hat,178,213,71,113,1,35,42,195,92,0.304688,0.054688,0.191667,0.087500
5,000002.jpg,1.jpg,hat,221,251,44,88,1,30,44,236,66,0.368750,0.046875,0.137500,0.091667
6,000002.jpg,1.jpg,hat,249,283,61,112,1,34,51,266,86,0.415625,0.053125,0.179167,0.106250
7,000002.jpg,1.jpg,hat,335,376,60,112,1,41,52,355,86,0.554688,0.064062,0.179167,0.108333
8,000002.jpg,1.jpg,hat,344,385,107,163,1,41,56,364,135,0.568750,0.064062,0.281250,0.116667
9,000002.jpg,1.jpg,hat,372,402,59,110,1,30,51,387,84,0.604688,0.046875,0.175000,0.106250


In [16]:
# First split: 80% train, 20% temporary (which will be split into validation and test)
df_train, df_temp = model_selection.train_test_split(df, test_size=0.2, random_state=13, shuffle=True)

# Second split: 50% of temporary set for validation and 50% for test (resulting in 10% each of the original dataset)
df_valid, df_test = model_selection.train_test_split(df_temp, test_size=0.5, random_state=13, shuffle=True)

# Output the sizes to verify the splits
print(f'Training set size: {len(df_train)}')
print(f'Validation set size: {len(df_valid)}')
print(f'Test set size: {len(df_test)}')

Training set size: 96448
Validation set size: 12056
Test set size: 12057


In [17]:
# creating directory for Train , Val and Test folders

# for images
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/train/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/valid/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/test/')

# for labels
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/train/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/valid/')
os.mkdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/test/')

In [18]:
# function to segregate_data
def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)

  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt")

    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(img_path,row.prev_filename), os.path.join(train_img_path,row.prev_filename))

In [20]:
## Apply function ##
src_img_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/raw_data/VOC2028/JPEGImages'
src_label_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/raw_data/VOC2028/Annotations'

train_img_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/train/'
train_label_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/train/'

valid_img_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/valid/'
valid_label_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/valid/'

test_img_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/test/'
test_label_path = '/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/test/'

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, src_label_path, valid_img_path, valid_label_path)
segregate_data(df_test, src_img_path, src_label_path, test_img_path, test_label_path)


In [21]:
print("No. of Training images", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/train/')))
print("No. of Training labels", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/train/')))

print("No. of valid images", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/valid/')))
print("No. of valid labels", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/valid/')))

print("No. of test images", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/images/test/')))
print("No. of test labels", len(os.listdir('/home/karush/code/karushp/project-helmet-det/helmet_det/data/data_split/labels/test/')))


No. of Training images 7212
No. of Training labels 7212
No. of valid images 3802
No. of valid labels 3802
No. of test images 3798
No. of test labels 3798
